In [1]:
import re

In [2]:
def process_cht_input(cheat, cheat_count, debug):
    cheat_output_partial = ""
    linecount = 0
    current_code_template=""
    codes_list_struct = []
    codes_description_global = ""
    single_patt = re.compile("^Codes \(single\)$")
    list_patt = re.compile("^Codes \(list\)$")
    code_patt = re.compile("^(.*) - .*$")
    desc_patt = re.compile("^.* - (.*)$")
    current_cheat_is_list, current_cheat_is_single = False, False
    
    for line in cheat.splitlines():
        if line == "" or line[0] == "#":
            if debug:
                print(str(linecount)+" Comment: "+line)
            linecount += 1
            continue
        
        if linecount == 0:
            codes_description_global = line
            if debug:
                print(str(linecount)+" Code description: "+codes_description_global)
            linecount += 1
            continue
        
        if "code template" in line:
            current_code_template = line
            if debug:
                print(str(linecount)+" Current Code Template: "+current_code_template)
            linecount += 1
            continue
        
        if single_patt.match(line):
            current_cheat_is_single = True
            linecount += 1
            continue
            
        if current_cheat_is_single:
            curr_cheat = {"Desc":codes_description_global,"Code":line}
            codes_list_struct.append(curr_cheat)
            #current_cheat_is_single = False
            linecount += 1
            continue
        
        if list_patt.match(line):
            current_cheat_is_list = True
            linecount += 1
            continue
        
        if current_cheat_is_list:
            if debug:
                print("enire list cheat line:"+line)
            cheat_code_str = code_patt.search(line).group(1).replace(" ","+")
            desc_str = codes_description_global+": "+desc_patt.search(line).group(1)
            
            if debug:
                print("cheat code str: "+cheat_code_str)
                print("desc str: "+desc_str+"\n")
            
            curr_cheat = {"Desc":desc_str,"Code":cheat_code_str}
            codes_list_struct.append(curr_cheat)
            linecount += 1
            continue # for completeness
            
    
    for idx,cheat_struct in enumerate(codes_list_struct):
        cheat_output_partial +='''
cheat'''+str(idx+cheat_count)+'''_desc = "'''+cheat_struct["Desc"]+'''"
cheat'''+str(idx+cheat_count)+'''_code = "'''+cheat_struct["Code"]+'''"
cheat'''+str(idx+cheat_count)+'''_enable = false
'''
    if debug:
        print(cheat_output_partial)
        
    return cheat_output_partial, len(codes_list_struct)

In [3]:
cht_file_str = ''''''
cheat_count = 0
debug = False
#change these variables to match your needs
cht_output_dir="./"
cht_output_fname="unbound.cht"
input_fname="./pokemon_unbound_cheat_input_formatted.txt"

with open(input_fname,'r') as data_file:
    split = data_file.read().split('----------\n')
    print("split length is: "+str(len(split)))
    for idx,cheat in enumerate(split):
        processed_cheat, curr_count = process_cht_input(cheat, cheat_count, debug)
        cheat_count += curr_count
        print("---- processed cheat block "+str(idx)+" ----")
        if debug:
            print(processed_cheat)
        cht_file_str+=processed_cheat
    print("total processed cheats: "+str(cheat_count))

In [4]:
cht_file_str = "cheats = "+str(cheat_count)+"\n" + cht_file_str

In [5]:
if debug: 
    print(cht_file_str)

In [6]:
cht_file = open(cht_output_dir+cht_output_fname, "w")
n = cht_file.write(cht_file_str)
cht_file.close()